In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

In [211]:
def merge_df(df):
    # 데이터 merge
    df = total_df[0].copy()
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + df['cat1'] + df['cat2'] + df['cat3']
    
    # 작업용 df로 변경
    df = df[['merge','keyword']]
    return df


def preprocessing(review, okt, remove_stopwords = False, stop_words = [], test = False):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if test:
        print(review_text)
        print(word_review)

    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]

    return word_review

## stop word
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']


# 리스트 재정리
def re_listing(info):
    total=[]
    for text in range(len(info)):
        temp = []
        for word in range(len(info[text])):
            if word == 0 :
                temp = info[text][word] 
            elif word < (len(info[text])):
                temp = temp  + ' ' + info[text][word] + ' ' 
            else:
                temp = temp + info[text][word]
        total.append(temp)
    return total

# 작업파일 불러오기

- 전체 :  ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시', '갤럭시케이스',
          '안경','모자','하이힐','후드','아이폰케이스','아이폰','자켓','청바지','점퍼','중고차','키보드','노트북','마우스','목도리',
          '네일','니트','목걸이','나이키','닌텐도','원피스','패딩','바지','향수','냉장고','레고','반지','셔츠','치마','스피커','정장',
          '선글라스','티셔츠','트레이닝','티비','지갑','세탁기','시계']
- '아디다스 0','에어컨 1','에어팟 2','오토바이 3','블라우스 4','팔찌 5','카메라 6','코트7','컴퓨터 8','구두 9','귀걸이10','갤럭시 11'

In [212]:
#내 작업
query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

In [213]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv'))

# 태그 추천 테스트
1. 문장-단어 TF-IDF 구하기 : 한 문장(product)에 중요도가 높은 단어들 테이블
2. 문장-문장 유사도 구하기 : 특정 문장(prodcut)기준 문장간 유사도를 통해 특정 문장과 유사한 문장 뽑기
3. 1을 미리 구해놓음 -> 2에서 비슷한 문장 뽑음 -> 2에서 뽑힌 문장을 1에서 찾음 -> 1에서 태그 뽑기


- 구현에 필요한 내용
 - 1,2번 구현 테스트
 - 언어 전처리 필요

In [214]:
adidas = merge_df(adidas)
adidas

,merge,keyword
0,아디다스 문디알 풋살화265m스포츠/레저축구축구 용품,"아디다스,문디알,코파문디알,아디다스문디알,풋살화"
1,아디다스 반팔티남성의류티셔츠반팔 티셔츠,"아디다스,아디다스반팔,반팔"
2,아디다스 봄가을 트레이닝 1230남성의류바지기타(바지),100
3,아디다스 아이바 글러브 레드 스포츠/레저기타 구기 스포츠0,"복싱,복싱글러브,아이바"
4,아디다스 빅로고 버건디 후드티여성의류후드티/후드집업0,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드"
...,...,...
5995,아디다스 x 고스티드+ 265 FG 최상급 축구화스포츠/레저축구축구 용품,"축구화,축구,아디다스x고스티드+,x고스티드,엑스"
5996,아디다스 유로파 핑크 하의 85여성의류트레이닝하의,"아디다스 유로파,유로파 85,유로파 핑크"
5997,아디다스 유로파 핑크 상의 85여성의류트레이닝상의,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상"
5998,<240>아디다스 오즈위고 스웨이드 화이트 새상품신발스니커즈0,"아디다스,오즈위고,240,EG0546,화이트"


## 불용어 처리

In [215]:
from konlpy.tag import Okt
import re

In [216]:
okt = Okt()
info=[]
for word in tqdm_notebook(adidas['merge']):
    info.append(preprocessing(word, okt, remove_stopwords = True, stop_words=stop_words))

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/4199161003.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm_notebook(adidas['merge']):


  0%|          | 0/6000 [00:00<?, ?it/s]

### 불용어 처리한 단어들 list 재정의

In [217]:
info = re_listing(info)
info

['아디다스 문디알  풋살  화  스포츠  레저  축구  축구  용품 ',
 '아디다스 반  팔  티  남성  의류  티셔츠  반  팔  티셔츠 ',
 '아디다스 봄  가을  트레이닝  남성  의류  바지  기타  바지 ',
 '아디다스 아이  바  글러브  레드  스포츠  레저  기타  구기  스포츠 ',
 '아디다스 빅로  고  버건디  후드  티  여  성  의류  후드  티  후드  집업 ',
 '아디다스 쪼리  신발  남성화  샌들  슬리퍼 ',
 '아디다스 니  짜다  운동화  신발  스니커즈 ',
 '아디다스 스니커즈  신발  스니커즈 ',
 '아디다스 크다  틀벨  스포츠  레저  헬스  요가  필라테스  헬스  요가  필라테스  장비 ',
 '라 프시  몬스  아디다스  테  렉스  판매  하다  신발  스니커즈 ',
 '아디다스 신발  신발  스니커즈 ',
 '아디다스 후  리스  팬츠  남성  의류  트레이닝  하의 ',
 '아디다스 운동화  신발  남성화  기타  남성화 ',
 '아디다스 스탠  스미스  신발  스니커즈 ',
 '팔라스 아디다스  트랙  팬츠  바지  판매  남성  의류  트레이닝  하의 ',
 '아디다스 유로파  남빨  져지  윤두준  져지  여성  의류  트레이닝  상의 ',
 '아디다스 키즈  용  반바지  유아  동  출산  여  의류  세  팬츠 ',
 '아디다스 노바  윈드브레이커  남성  의류  트레이닝  상의 ',
 '아디다스 깔깔이  패딩  네이비  점퍼  남성  의류  패딩  점퍼  기타  패딩  점퍼 ',
 '파워 정품  아코  탭  미시  착  새  상품  아디다스  오  즈  위고  최저  신발  스니커즈 ',
 '매 물정  품  대장  급  아디다스  라지  누빔  점퍼  자켓  팔라스  맛  최저  남성  의류  패  딩점퍼롱패딩 ',
 '뉴발란스 레인보우  한정판  신발  스니커즈 ',
 '아디다스 프리미엄  다운  패딩  판매  하다  남성  의류  패  딩점퍼숏패딩 ',
 '아디다스 후드  

### product별 tf-idf 구하기

In [219]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = adidas['product_id']
countvect_df

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하단,하드,하든,하메스,하복,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 10000)
tfvect = vect.fit(info)

tfidv_df = pd.DataFrame(tfvect.transform(info).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = adidas['product_id']
tfidv_df

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하단,하드,하든,하메스,하복,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### 비슷한 prodcut 추출

In [224]:
cosine_matrix = cosine_similarity(tfidv_df,tfidv_df)
cosine_matrix.shape

(6000, 6000)

In [225]:
product_df = pd.DataFrame(cosine_matrix, columns = adidas['product_id'])
product_df.index = adidas['product_id']
product_df

KeyError: 'product_id'

### top3 상품별 중요도 높은 키워드 top5 고르기

In [ ]:
# 특정 상품지정 및 유사한 product 3개 뽑기
top3 = product_df.iloc[2].sort_values(ascending=False)[1:4].index

# product_id str로 변환
product_df.index = product_df.index.astype(str)
tfidv_df.index = tfidv_df.index.astype(str)

# top3 상품의 연관성 높은 키워드 top5 뽑기
total_tag =[]
for i in range(len(top3)):
    tfidv_df[tfidv_df.index == str(top3[i])]
    temp =[]
    # 유사 상품중 top5 키워드 추천
    temp.append(tfidv_df[tfidv_df.index == str(top3[i])].T.sort_values(by = str(top3[i]), ascending=False)[:5].index.to_list())

    # top5 중복 제거
    for i in range(len(temp[0])):
        total_tag.append(temp[0][i])
total_tag = list(set(total_tag))
total_tag

In [ ]:
top3

# 방향성 재정의
- 제목 + 카테고리 = feature
- 태그 = label
- feature + label Matrix를 train, test로 구분
- train set 각 태그별 feature 유사도 Matrix 생성
- test set feature를 통한 label 예측으로 평가지표 생성
+ 태그별 유사도 뽑는 과정 구체화 필요
+ feature 넣었을 때 유사도를 어떤식으로 취합해서 태그를 뽑을지 구체화 필요
+ 태그 순위 개수를 지정하는 것이 좋을지, 그렇게 한다면 상위 Top10 태그를 미리 생성해서 추천한것 + Top10 태그로 태그 5개 추천 사이즈 맞추기 등 구체화

### feature, label 구분

In [226]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv'))

In [227]:
adidas = total_df[0]
adidas = merge_df(adidas)
adidas.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
adidas

,feature,label
0,아디다스 문디알 풋살화265m스포츠/레저축구축구 용품,"아디다스,문디알,코파문디알,아디다스문디알,풋살화"
1,아디다스 반팔티남성의류티셔츠반팔 티셔츠,"아디다스,아디다스반팔,반팔"
2,아디다스 봄가을 트레이닝 1230남성의류바지기타(바지),100
3,아디다스 아이바 글러브 레드 스포츠/레저기타 구기 스포츠0,"복싱,복싱글러브,아이바"
4,아디다스 빅로고 버건디 후드티여성의류후드티/후드집업0,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드"
...,...,...
5995,아디다스 x 고스티드+ 265 FG 최상급 축구화스포츠/레저축구축구 용품,"축구화,축구,아디다스x고스티드+,x고스티드,엑스"
5996,아디다스 유로파 핑크 하의 85여성의류트레이닝하의,"아디다스 유로파,유로파 85,유로파 핑크"
5997,아디다스 유로파 핑크 상의 85여성의류트레이닝상의,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상"
5998,<240>아디다스 오즈위고 스웨이드 화이트 새상품신발스니커즈0,"아디다스,오즈위고,240,EG0546,화이트"


### label 부터

In [228]:
# label 가장 많은 것 확인
long = 0
for i in range(len(adidas)):
    if len(adidas.label[i].split(',')) > long:
        long = len(adidas.label[i].split(','))
long

5

In [288]:
# lable 5개 각각 columns 으로 지정
for i in range(1,6):
    adidas['label' + str(i)] = 0
for i in range(len(adidas)):
    try:
        adidas.label1[i] = adidas.label[i].split(',')[0]
    except: continue
    try:
        adidas.label2[i] = adidas.label[i].split(',')[1]
    except: continue
    try:
        adidas.label3[i] = adidas.label[i].split(',')[2]
    except: continue
    try:
        adidas.label4[i] = adidas.label[i].split(',')[3]
    except: continue
    try:
        adidas.label5[i] = adidas.label[i].split(',')[4]
    except: continue
adidas_label = adidas[['label1','label2','label3','label4','label5']]
adidas_label

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/585579372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adidas.label1[i] = adidas.label[i].split(',')[0]
/Users/ppangppang/opt/anaconda3/envs/ppangppang/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/585579372.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,label1,label2,label3,label4,label5
0,아디다스,문디알,코파문디알,아디다스문디알,풋살화
1,아디다스,아디다스반팔,반팔,0,0
2,100,0,0,0,0
3,복싱,복싱글러브,아이바,0,0
4,아디다스빅로고,아디다스후드티,빅로고후드,버건디후드,0
...,...,...,...,...,...
5995,축구화,축구,아디다스x고스티드+,x고스티드,엑스
5996,아디다스 유로파,유로파 85,유로파 핑크,0,0
5997,아디다스 유로파,85,유로파 핑크,아디다스 여자 상,0
5998,아디다스,오즈위고,240,EG0546,화이트


In [231]:
okt = Okt()
info=[]
for word in tqdm_notebook(adidas['feature']):
    info.append(preprocessing(word, okt, remove_stopwords = True, stop_words=stop_words))
info = re_listing(info)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 10000)
tfvect = vect.fit(info)

tfidv_df = pd.DataFrame(tfvect.transform(info).toarray(), columns = sorted(vect.vocabulary_))
# tfidv_df.index = adidas['product_id']
tfidv_df

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/2662520959.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm_notebook(adidas['feature']):


  0%|          | 0/6000 [00:00<?, ?it/s]

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하단,하드,하든,하메스,하복,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [232]:
adidas_df = tfidv_df
adidas_df[['label1','label2','label3','label4','label5']] = adidas_label[['label1','label2','label3','label4','label5']]
adidas_df

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색,label1,label2,label3,label4,label5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,아디다스,문디알,코파문디알,아디다스문디알,풋살화
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,아디다스,아디다스반팔,반팔,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.675109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.439238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [276]:
adidas

,feature,label,label1,label2,label3,label4,label5
0,아디다스 문디알 풋살화265m스포츠/레저축구축구 용품,"아디다스,문디알,코파문디알,아디다스문디알,풋살화",아디다스,문디알,코파문디알,아디다스문디알,풋살화
1,아디다스 반팔티남성의류티셔츠반팔 티셔츠,"아디다스,아디다스반팔,반팔",아디다스,아디다스반팔,반팔,0,0
2,아디다스 봄가을 트레이닝 1230남성의류바지기타(바지),100,100,0,0,0,0
3,아디다스 아이바 글러브 레드 스포츠/레저기타 구기 스포츠0,"복싱,복싱글러브,아이바",복싱,복싱글러브,아이바,0,0
4,아디다스 빅로고 버건디 후드티여성의류후드티/후드집업0,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드",아디다스빅로고,아디다스후드티,빅로고후드,버건디후드,0
...,...,...,...,...,...,...,...
5995,아디다스 x 고스티드+ 265 FG 최상급 축구화스포츠/레저축구축구 용품,"축구화,축구,아디다스x고스티드+,x고스티드,엑스",축구화,축구,아디다스x고스티드+,x고스티드,엑스
5996,아디다스 유로파 핑크 하의 85여성의류트레이닝하의,"아디다스 유로파,유로파 85,유로파 핑크",아디다스 유로파,유로파 85,유로파 핑크,0,0
5997,아디다스 유로파 핑크 상의 85여성의류트레이닝상의,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상",아디다스 유로파,85,유로파 핑크,아디다스 여자 상,0
5998,<240>아디다스 오즈위고 스웨이드 화이트 새상품신발스니커즈0,"아디다스,오즈위고,240,EG0546,화이트",아디다스,오즈위고,240,EG0546,화이트


In [273]:
temp = pd.DataFrame()
for i in range(len(adidas)):
    if adidas['label1'][i] == '아디다스':
        temp = pd.concat([temp, adidas.iloc[[i]]])
    elif adidas['label2'][i] == '아디다스':
        temp = pd.concat([temp, adidas.iloc[[i]]])
    elif adidas['label3'][i] == '아디다스':
        temp = pd.concat([temp, adidas.iloc[[i]]])
    elif adidas['label4'][i] == '아디다스':
        temp = pd.concat([temp, adidas.iloc[[i]]])
    elif adidas['label5'][i] == '아디다스':
        temp = pd.concat([temp, adidas.iloc[[i]]])
temp = temp[['feature','label']]
temp['label'] = '아디다스'
temp

,feature,label
0,아디다스 문디알 풋살화265m스포츠/레저축구축구 용품,아디다스
1,아디다스 반팔티남성의류티셔츠반팔 티셔츠,아디다스
5,아디다스 쪼리신발남성화샌들/슬리퍼,아디다스
6,아디다스 니짜 운동화신발스니커즈0,아디다스
7,아디다스 스니커즈 240신발스니커즈0,아디다스
...,...,...
5991,아다다스 클라이마라이트 기능성 반팔남성의류티셔츠반팔 티셔츠,아디다스
5992,아디다스 네메시스.1 ag 265스포츠/레저축구축구 용품,아디다스
5993,아디다스 후드티(남여공용)남성의류트레이닝상의,아디다스
5998,<240>아디다스 오즈위고 스웨이드 화이트 새상품신발스니커즈0,아디다스


In [274]:
okt = Okt()
info=[]
for word in tqdm_notebook(temp['feature']):
    info.append(preprocessing(word, okt, remove_stopwords = True, stop_words=stop_words))
info = re_listing(info)

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 10000)
tfvect = vect.fit(info)

tfidv_df = pd.DataFrame(tfvect.transform(info).toarray(), columns = sorted(vect.vocabulary_))
# tfidv_df.index = adidas['product_id']
tfidv_df

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/2467169962.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm_notebook(temp['feature']):


  0%|          | 0/4115 [00:00<?, ?it/s]

,ㄷㅅ,ㅠㅠ,가격,가능,가드,가디건,가바,가방,가볍다,가슴,가정,가젤,가죽,가토,가트,간지,강동원,강렬하다,강민호,강아지,강추,같다,개다,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,검검,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공홈,교신,교육,교환,구기,구두,구매,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,구입,구제,구찌,구합,구해,국가대표,국내,굳다,권투,귀걸이,귀엽다,규어,그래픽,그레이,그로,그리드,그린,극미,글라이드,글로리,글리,금색,급매,...,피그먼트,피나,피날레,피다,피렐리,피버,피스,피어,피오,피쿠시,피탈,피터,필기,필드,필라,필라테스,필름,필수,핏더,핑크,핑크색,핑크팬더,하남성,하늘색,하다,하든,하메스,하복,하야시,하우스,하의,하이,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학습,한국,한빈,한센,한일,한정,한정판,한파,할인,핫팬츠,핫핑크,항공,해골,해보다,해외,해찬,핸더,햡니,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,화이트,환영,황금,회남,회색,회착,후드,후디,후디드,휠라,휴먼,희귀,희다,흰색,흰핑,히로,히트,히피,힐탑,힙색
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,

In [281]:
tfidv_df['label'] = temp['label']
tf_test = tfidv_df.groupby('label').mean()
tf_test

,ㄷㅅ,ㅠㅠ,가격,가능,가드,가디건,가바,가방,가볍다,가슴,가정,가젤,가죽,가토,가트,간지,강동원,강렬하다,강민호,강아지,강추,같다,개다,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,검검,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공홈,교신,교육,교환,구기,구두,구매,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,구입,구제,구찌,구합,구해,국가대표,국내,굳다,권투,귀걸이,귀엽다,규어,그래픽,그레이,그로,그리드,그린,극미,글라이드,글로리,글리,금색,급매,...,피그먼트,피나,피날레,피다,피렐리,피버,피스,피어,피오,피쿠시,피탈,피터,필기,필드,필라,필라테스,필름,필수,핏더,핑크,핑크색,핑크팬더,하남성,하늘색,하다,하든,하메스,하복,하야시,하우스,하의,하이,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학습,한국,한빈,한센,한일,한정,한정판,한파,할인,핫팬츠,핫핑크,항공,해골,해보다,해외,해찬,핸더,햡니,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,화이트,환영,황금,회남,회색,회착,후드,후디,후디드,휠라,휴먼,희귀,희다,흰색,흰핑,히로,히트,히피,힐탑,힙색
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
아디다스,0.000167,0.000151,0.005214,0.001229,0.000823,0.000558,0.0,0.009764,0.0,0.000733,0.0,0.002022,0.001367,0.00032,0.0,0.000133,0.000276,0.000149,0.000154,0.000289,0.0,0.000178,0.000321,0.0,0.00017,0.000446,0.000148,0.000398,0.000471,0.000136,0.001242,0.00029,0.000193,0.0,0.000227,0.000281,0.000699,0.0,0.000244,0.0,0.000151,0.000348,0.00012,0.002432,0.00031,0.000117,0.00127,0.000095,0.000319,0.000252,0.000681,0.000553,0.000192,0.000255,0.001823,0.000416,0.000453,0.000832,0.000137,0.000134,0.000366,0.00705,0.000172,0.0,0.002741,0.0,0.001143,0.000141,0.000324,0.001099,0.000638,0.000884,0.000174,0.000452,0.0,0.0,0.000214,0.000288,0.001671,0.000201,0.0,0.000413,0.0003,0.000694,0.000142,0.00018,0.000238,0.0,0.000166,0.00097,0.003572,0.00017,0.000201,0.00052,0.000353,0.000259,0.000203,0.0,0.000151,0.00035,...,0.0,0.0,0.000316,0.000084,0.000266,0.0,0.00015,0.0,0.000358,0.0,0.000198,0.000223,0.00015,0.000447,0.000532,0.00328,0.000109,0.0,0.0,0.00179,0.0,0.000173,0.000219,0.0,0.006465,0.00013,0.000209,0.000746,0.0,0.000117,0.007554,0.002404,0.000377,0.000207,0.0,0.000323,0.000297,0.000843,0.00015,0.000208,0.000166,0.00015,0.0,0.000224,0.003739,0.0,0.000796,0.000097,0.00029,0.001646,0.000241,0.000136,0.000798,0.000189,0.000161,0.0,0.000296,0.0,0.000174,0.000128,0.0,0.00074,0.000368,0.000218,0.000216,0.0,0.0004,0.00015,0.003352,0.000073,0.000168,0.000188,0.001067,0.0,0.000232,0.000198,0.000121,0.0,0.000379,0.0,0.005244,0.000211,0.000442,0.000216,0.000791,0.000144,0.048583,0.001387,0.000181,0.001521,0.001206,0.000796,0.003131,0.001076,0.000159,0.000133,0.000212,0.0,0.000122,0.000366


In [286]:
test_df = adidas_df[adidas_df.columns[:-5]]
for col in test_df.columns:
    test_df[col].values[:] = 0
test_df = test_df[:0]
test_df

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하단,하드,하든,하메스,하복,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색


In [287]:
pd.concat([test_df,tf_test])

,ㄷㅅ,ㅠㅠ,가격,가격파괴,가구,가능,가드,가디건,가바,가방,가볍다,가슴,가오리,가을,가전제품,가정,가젤,가죽,가지,가토,가트,각종,간절,간지,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거리,거여,거위,거유,거의,건담,건빵,검검,검다,검베,검빨,검솔,검은색,검정,검파,검핑,겉옷,게스,게임,겐조,겨울,격인,경기,경량,경북,경찰차,경희대학교,계기,계시,고급,고급스럽다,고베,고샤,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,골프장,공예품,공용,공인,공홈,광고,교복,교신,교육,교환,구기,구기다,구도,구두,구매,구미,구성,구스,구스다운롱패딩,구스다운패딩,구스패딩,...,하단,하드,하든,하메스,하복,하야시,하얗다,하우스,하의,하이,하이라이트,하이브리드,하이퍼,하이퍼스페이스,하키,하트,하프,학생운동,학습,한국,한번,한빈,한센,한일,한일월드컵,한정,한정판,한파,할인,함부르크,핫딜,핫팬츠,핫핑크,항공,해골,해보다,해쉬,해외,해찬,핸더,핸드,햇빛,햡니,향수,허니,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬싱키,현대,형광,호날두,호랑이,호스,호텔,홀로그램,홀리스,홈웨어,홍보,화구,화이트,환영,활동,황금,회남,회색,회실착,회착,회핑,후드,후디,후디드,후루룩,후리,후망,후유,후집,휠라,휴먼,흑금,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색
아디다스,0.000167,0.000151,0.005214,NaN,NaN,0.001229,0.000823,0.000558,0.0,0.009764,0.0,0.000733,NaN,NaN,NaN,0.0,0.002022,0.001367,NaN,0.00032,0.0,NaN,NaN,0.000133,NaN,NaN,NaN,NaN,0.000276,0.000149,0.000154,0.000289,0.0,0.000178,0.000321,NaN,0.0,0.00017,0.000446,0.000148,0.000398,0.000471,0.000136,0.001242,0.00029,NaN,0.000193,NaN,NaN,0.0,0.000227,0.000281,0.000699,0.0,0.000244,0.0,0.000151,0.000348,0.00012,0.002432,0.00031,0.000117,0.00127,0.000095,0.000319,0.000252,0.000681,0.000553,NaN,0.000192,0.000255,0.001823,0.000416,0.000453,0.000832,0.000137,0.000134,0.000366,0.00705,0.000172,0.0,0.002741,NaN,0.0,NaN,NaN,0.001143,0.000141,0.000324,0.001099,NaN,NaN,0.000638,0.000884,NaN,0.000174,0.000452,0.0,0.0,0.000214,...,NaN,NaN,0.00013,0.000209,0.000746,0.0,NaN,0.000117,0.007554,0.002404,NaN,0.000377,0.000207,0.0,0.000323,0.000297,0.000843,NaN,0.00015,0.000208,NaN,0.000166,0.00015,0.0,NaN,0.000224,0.003739,0.0,0.000796,NaN,NaN,0.000097,0.00029,0.001646,0.000241,0.000136,NaN,0.000798,0.000189,0.000161,NaN,NaN,0.0,NaN,0.000296,0.0,0.000174,0.000128,0.0,0.00074,0.000368,0.000218,0.000216,0.0,0.0004,0.00015,0.003352,0.000073,0.000168,0.000188,0.001067,0.0,0.000232,0.000198,0.000121,0.0,0.000379,0.0,NaN,NaN,0.005244,0.000211,NaN,0.000442,0.000216,0.000791,NaN,0.000144,NaN,0.048583,0.001387,0.000181,NaN,NaN,NaN,NaN,NaN,0.001521,0.001206,NaN,0.000796,0.003131,0.001076,0.000159,0.000133,NaN,0.000212,0.0,0.000122,0.000366
